Here, we are just trying to test 'Functional Models' from TF.

In [75]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import time
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import keras as K
from tensorflow import keras
#from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from keras.layers import Dense


In [76]:

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [77]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

In [78]:
#normalizing the datasets in the range of 0-1
train_images = train_images/255.0
test_images = test_images/255.0

Defining our model

In [82]:
app = VGG16
init_model = app(include_top = False,
            weights = "imagenet",
            input_shape = (32,32,3),
            input_tensor = None)
init_model.trainable = False

inputs = keras.Input(shape=(32, 32, 3))
# # making sure that the base_model is running in inference mode here, by passing `training=False`.
x = init_model(inputs, training=False)

DANN (Domain-Adversarial Neural Networks)

In [83]:
#here we are using Feature-based Domain Adaptation Technique
# here we try to distinguish between source and target domain data, forcing the model to learn domain-invariant features by adding and training new TOP_LAYERS

x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier - ten classes
outputs = (Dense(10, activation='softmax'))(x)
model = keras.Model(inputs, outputs)

In [84]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
# Train the model and display the activations after each epoch
start_time = time.time()
model.fit(train_images, train_labels, epochs=1, validation_data=(test_images, test_labels))
end_time = time.time()

1563/1563 [==============================] - 152s 96ms/step - loss: 1.5365 - accuracy: 0.4760 - val_loss: 1.3753 - val_accuracy: 0.5276


In [87]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

313/313 [==============================] - 24s 77ms/step - loss: 1.3753 - accuracy: 0.5276
Test accuracy: 0.5275999903678894
Time elapsed:  153.43144702911377


In [98]:
model.layers